<a href="https://colab.research.google.com/github/nzcbass/Pythontoopenai/blob/main/analysingJsonoutputfromCVparser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
import json

# Assuming 'resume.json' is in a directory called 'resumes' in the same parent directory as your Python script
json_file_path1 = "/content/CV_parsed_by_CV_Parser_Premium-6.json"  # Relative path

# Open the JSON file and load its contents into a Python dictionary
with open(json_file_path1, 'r') as file:
    resume_data = json.load(file)

# Labeling the job entries
job_count = 1
for experience in resume_data['profile']['professional_experiences']:
    experience['job_label'] = f'Job_{job_count}'
    job_count += 1

# Save the modified data back to the JSON file
output_json_path = "/content/jobslabeled.json"
with open(output_json_path, 'w') as output_file:
    json.dump(resume_data, output_file, indent=2)

print(f"Job labeling completed. Labeled data saved to {output_json_path}")

# Now 'resume_data' contains the contents of the JSON file
# print(json.dumps(resume_data, indent=2))


Job labeling completed. Labeled data saved to /content/jobslabeled.json


In [148]:
import json
import pandas as pd

def json_to_dataframe(json_file_path):
    # Open the JSON file and load its contents into a Python dictionary
    with open(json_file_path, 'r') as file:
        resume_data = json.load(file)

    # Convert the labeled data into a DataFrame
    df = pd.json_normalize(resume_data['profile']['professional_experiences'])

    # Remove the 'description' column if it exists e.g. candidate blurb on the roles they've had
    if 'description' in df.columns:
        df = df.drop('description', axis=1)

    return df

if __name__ == "__main__":
    # Assuming 'jobslabeled.json' is in the same directory as your new script
    json_file_path2 = "jobslabeled.json"

    # Convert the labeled JSON data to a DataFrame
    df = json_to_dataframe(json_file_path2)

    # makes location lower case
    df['location'] = df['location'].str.lower()

    # Keep only the last 2 words from the right-hand side
    df['location'] = df['location'].str.split().apply(lambda x: ' '.join(x[-2:]) if len(x) > 1 else ' '.join(x))

    # Remove the word 'city' from the 'location' column in df and remove numbers
    df['location'] = df['location'].str.replace(r'\b(city)\b|\d', '', case=False, regex=True)

    # Remove anything except text and spaces from the 'location' column
    # df['location'] = df['location'].replace(to_replace=r'[^a-zA-Z\s]', value='', regex=True)

    # Remove anything from the 'Location' column except text and spaces between words
    df['location'] = df['location'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

    # Remove leading and trailing whitespaces from the strings in the 'Location' column
    df['location'] = df['location'].str.strip()

    # Replacing blanks in Location with NA
    df['location'].replace('', pd.NA, inplace=True)

    # Save the DataFrame as a CSV file
    csv_output_path = "output_data.csv"
    df.to_csv(csv_output_path, index=False)

    # Display the DataFrame
    print(df)

   is_current  duration_in_months                  company location  \
0        True                  14      OCS 1rst mechanical     <NA>   
1       False                  17          TURBO STAFF LTD     <NA>   
2       False                  13              PEC PTY LTD     <NA>   
3       False                  15  HI-TEC WELDING PTY LTD.     <NA>   
4       False                 104          EEI CORPORATION     <NA>   

                         title job_label  start_date.year  start_date.month  \
0              Pipefitter/HVAC     Job_1             2022                10   
1           PIPEFITTER/PLUMBER     Job_2             2019                 5   
2                   PIPEFITTER     Job_3             2016                11   
3  PIPEFITTER/METAL FABRICATOR     Job_4             2015                 8   
4                   PIPEFITTER     Job_5             2006                 4   

   end_date.year  end_date.month  
0           2023              11  
1           2020            

In [126]:
pip install dataprep

In [149]:
from dataprep.clean import clean_country
import pandas as pd

# # Clean the 'location' column using clean_country
cleaned_df = clean_country(df, "location", input_format="official")
df = cleaned_df


# # Display the cleaned DataFrame
print(df)

# # output is location_cleaned column



/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7084: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(
                                     

Country Cleaning Report:
	5 values unable to be parsed (100.0%), set to NaN
Result contains 0 (0.0%) values in the correct format and 5 null values (100.0%)
   is_current  duration_in_months                  company location  \
0        True                  14      OCS 1rst mechanical     <NA>   
1       False                  17          TURBO STAFF LTD     <NA>   
2       False                  13              PEC PTY LTD     <NA>   
3       False                  15  HI-TEC WELDING PTY LTD.     <NA>   
4       False                 104          EEI CORPORATION     <NA>   

                         title job_label  start_date.year  start_date.month  \
0              Pipefitter/HVAC     Job_1             2022                10   
1           PIPEFITTER/PLUMBER     Job_2             2019                 5   
2                   PIPEFITTER     Job_3             2016                11   
3  PIPEFITTER/METAL FABRICATOR     Job_4             2015                 8   
4                   P

In [151]:
import pandas as pd
import re

xlsx_file_path = "/content/Updated_World_Cities.xlsx"

# Assuming your DataFrame is named df, make sure to read or define it before applying operations
# df = pd.read_csv(csv_file_path)  # Uncomment and replace with your DataFrame creation if needed

# Read the Excel file into a DataFrame
df1 = pd.read_excel(xlsx_file_path)

# Convert 'location' and 'City' columns to lowercase in df
df['location'] = df['location'].str.lower()

# Remove the word 'city' from the 'City' column in df1 and remove numbers
df1['City'] = df1['City'].str.replace(r'\b(city)\b|\d', '', case=False, regex=True)

# Replace multiple spaces with a single space in the 'location' column of df
df['location'] = df['location'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Create a mapping between 'City' and 'Country' columns in df1
mapping_city_country = dict(zip(df1['City'], df1['Country']))

# Custom function to find matches in 'location' column
def find_city(location):
    for city in mapping_city_country.keys():
        if pd.notna(location) and re.search(rf'\b{re.escape(city)}\b', location, flags=re.IGNORECASE):
            return city
    return pd.NA

# Apply the custom function to find matching cities
df['extracted_city'] = df['location'].apply(find_city)

# Use the extracted city to map the corresponding country
df['country_matched_city'] = df['extracted_city'].map(mapping_city_country)

# Create a new column 'combined_location'
df['combined_location'] = df['location_clean'].combine_first(df['country_matched_city'])

# Capitalize the first letter of each word in 'combined_location'
df['combined_location'] = df['combined_location'].apply(lambda x: x.title() if pd.notna(x) else pd.NA)

# Apply the uppercase rule for words with 3 letters or less
df['combined_location'] = df['combined_location'].apply(lambda x: x.upper() if pd.notna(x) and len(x) <= 3 else x)

# Drop intermediate columns if needed
df.drop(['extracted_city', 'country_matched_city'], axis=1, inplace=True)

# Display the updated DataFrame
print(df)


   is_current  duration_in_months                  company location  \
0        True                  14      OCS 1rst mechanical     <NA>   
1       False                  17          TURBO STAFF LTD     <NA>   
2       False                  13              PEC PTY LTD     <NA>   
3       False                  15  HI-TEC WELDING PTY LTD.     <NA>   
4       False                 104          EEI CORPORATION     <NA>   

                         title job_label  start_date.year  start_date.month  \
0              Pipefitter/HVAC     Job_1             2022                10   
1           PIPEFITTER/PLUMBER     Job_2             2019                 5   
2                   PIPEFITTER     Job_3             2016                11   
3  PIPEFITTER/METAL FABRICATOR     Job_4             2015                 8   
4                   PIPEFITTER     Job_5             2006                 4   

   end_date.year  end_date.month  location_clean combined_location  
0           2023             

In [ ]:
# import pandas as pd

# xlsx_file_path = "/content/Updated_World_Cities.xlsx"

# # Assuming your DataFrame is named df, make sure to read or define it before applying operations
# # df = pd.read_csv(csv_file_path)  # Uncomment and replace with your DataFrame creation if needed

# # Read the Excel file into a DataFrame
# df1 = pd.read_excel(xlsx_file_path)

# # Convert 'location' and 'City' columns to lowercase in both DataFrames
# df['location'] = df['location'].str.lower()
# df1['City'] = df1['City'].str.lower()

# # Remove the word 'city' from the 'City' column in df1
# df1['City'] = df1['City'].str.replace(r'\b(city)\b', '', case=False)

# # Assuming your DataFrame is named df
# # df['location'] = df['location'].str.replace(r'\b\w*-\w*-\w*\b', '', regex=True)  # Uncomment if needed

# # Create a mapping between 'City', 'iso2', and 'iso3' columns in df1
# mapping_city_country = dict(zip(df1['City'], df1['Country']))
# mapping_iso2 = dict(zip(df1['iso2'], df1['Country']))
# mapping_iso3 = dict(zip(df1['iso3'], df1['Country']))

# # Create a pattern for regex using city names
# pattern_city = '|'.join([rf'\b{city}\b' for city in df1['City']])
# pattern_iso2 = '|'.join([rf'\b{iso2}\b' for iso2 in df1['iso2']])
# pattern_iso3 = '|'.join([rf'\b{iso3}\b' for iso3 in df1['iso3']])

# # Use str.contains with regex pattern to check for city names in 'location' column
# df['contains_city'] = df['location'].str.contains(pattern_city, case=False, na=False, regex=True)

# # Use str.contains with regex pattern to check for iso2 and iso3 in 'location' column
# df['contains_iso2'] = df['location'].str.contains(pattern_iso2, case=False, na=False, regex=True)
# df['contains_iso3'] = df['location'].str.contains(pattern_iso3, case=False, na=False, regex=True)

# # If a city is found, map the corresponding country using the mapping
# df['country_matched_city'] = df['location'].apply(
#     lambda x: next((country for city, country in mapping_city_country.items() if pd.notna(x) and city in x), pd.NA)
# )

# # If iso2 is found, map the corresponding country using the mapping
# df['country_matched_iso2'] = df['location'].map(mapping_iso2)

# # If iso3 is found, map the corresponding country using the mapping
# df['country_matched_iso3'] = df['location'].map(mapping_iso3)

# # Create a new column 'combined_location'
# df['combined_location'] = df.apply(
#     lambda row: row['location_clean'] if pd.notna(row['location_clean']) else row['country_matched_city'],
#     axis=1
# )

# # Capitalize the first letter of each word in 'combined_location'
# df['combined_location'] = df['combined_location'].apply(lambda x: x.title() if pd.notna(x) else pd.NA)

# # Apply the uppercase rule for words with 4 letters or less
# df['combined_location'] = df['combined_location'].apply(lambda x: x.upper() if pd.notna(x) and len(x) <= 4 else x)

# # Display the updated DataFrame
# print(df)


<ipython-input-53-de95332a8d55>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['City'] = df1['City'].str.replace(r'\b(city)\b', '', case=False)
<ipython-input-53-de95332a8d55>:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['contains_city'] = df['location'].str.contains(pattern_city, case=False, na=False, regex=True)


   is_current  duration_in_months  \
0        True                  56   
1       False                  22   
2       False                   2   
3       False                  33   
4       False                   4   
5       False                  24   
6       False                  35   

                                             company  \
0                                 1st MECHANICAL Ltd   
1                                        TURBO STAFF   
2                                          A.W.F LTD   
3                                          LAKESMARE   
4              LCPI LEIHGTON CONTRUCTORS PHIL. CORP.   
5  RALGO INDUSTRIES INC. AIRCONDITIONING AND REFR...   
6                            TARGERC AIRCONDITIONING   

                                            location                   title  \
0  17 greenpark road, penrose, auckland new zeala...             DUCT FITTER   
1                          4 manu tapo drive mangere             DUCT FITTER   
2             